# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [117]:
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [118]:
df = pd.read_csv(r".\data\Admission_Predict.csv")
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Remove trailing spaces at the end of the column names if there are any.


In [119]:
df.columns = df.columns.str.rstrip()
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
...,...,...,...,...,...,...,...,...,...
380,381,324,110,3,3.5,3.5,9.04,1,0.82
381,382,325,107,3,3.0,3.5,9.11,1,0.84
382,383,330,116,4,5.0,4.5,9.45,1,0.91
383,384,312,103,3,3.5,4.0,8.78,0,0.67


Let's evaluate the dataset by looking at the `head` function.

In [120]:
df.head(10)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90
5,6,321,109,3,3.0,4.0,8.20,1,0.75
6,7,308,101,2,3.0,4.0,7.90,0,0.68
7,8,302,102,1,2.0,1.5,8.00,0,0.50
8,9,323,108,3,3.5,3.0,8.60,0,0.45
9,10,327,111,4,4.0,4.5,9.00,1,0.84


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [121]:
print(df.isnull().sum())

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64


Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [122]:
df.set_index('Serial No.', inplace=True)
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [123]:
df[["GRE Score", "CGPA"]].value_counts()

GRE Score  CGPA
290        7.46    1
317        8.56    1
323        9.10    1
           8.98    1
           8.88    1
                  ..
312        8.02    1
           8.01    1
           7.90    1
311        9.00    1
340        9.92    1
Length: 385, dtype: int64

Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [124]:
df.set_index(['GRE Score', 'CGPA'], inplace=True)

df.reset_index(inplace=True)

df.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
1,316,8.00,104,3,3.0,3.5,1,0.72
2,322,8.67,110,3,3.5,2.5,1,0.80
3,314,8.21,103,2,2.0,3.0,0,0.65
4,330,9.34,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [125]:
num_rows = len(df[(df['CGPA'] > 9) & (df['Research'] == 1)])
print(f"There are {num_rows} rows where the CGPA is higher than 9 and the student has conducted research.")

There are 101 rows where the CGPA is higher than 9 and the student has conducted research.


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [126]:
chance_of = df[(df["CGPA"] > 9) & (df["SOP"]<3.5)]["Chance of Admit"].mean()
chance_of

0.8019999999999999

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [127]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    result = ((col - col.mean()) / col.std())
    return result   
    # your code here

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [128]:
df["CGPA_std"] = standardize(df["CGPA"])
df["GRE_std"] = standardize(df["GRE Score"])
df["LOR_std"] = standardize(df["LOR"])

df.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713
4,330,9.34,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713


We will generate the decision choice at random using the code below. Please run the cell.

In [129]:
# Libraries
from random import choices

In [130]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=df.shape[0])

Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [131]:
df['deciding_column'] = df.lookup(df.index, decision_choice)

C:\Users\jmpcs\AppData\Local\Temp\ipykernel_25264\2449800145.py:1: FutureWarning: The 'lookup' method is deprecated and will be removed in a future version. You can use DataFrame.melt and DataFrame.loc as a substitute.
  df['deciding_column'] = df.lookup(df.index, decision_choice)


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [132]:
df.loc[df['deciding_column'] > 0.8, 'decision'] = 1
df.loc[df['deciding_column'] <= 0.8, 'decision'] = 0

df

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,118,4,4.5,4.5,1,0.92,1.747900,1.753381,1.191647,1.753381,1.0
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.991211,-0.063367,0.076740,-0.991211,0.0
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121034,0.455704,-1.038166,-1.038166,0.0
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.642597,-0.236391,-0.480713,-0.236391,0.0
4,330,9.34,115,5,4.5,3.0,1,0.90,1.233279,1.147798,-0.480713,1.147798,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,324,9.04,110,3,3.5,3.5,1,0.82,0.735259,0.628727,0.076740,0.628727,0.0
381,325,9.11,107,3,3.0,3.5,1,0.84,0.851463,0.715239,0.076740,0.076740,0.0
382,330,9.45,116,4,5.0,4.5,1,0.91,1.415886,1.147798,1.191647,1.147798,1.0
383,312,8.78,103,3,3.5,4.0,0,0.67,0.303641,-0.409414,0.634193,0.303641,0.0


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [133]:
(df['decision'] > 0).sum()

92

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [134]:
df.columns = df.columns.str.replace('[^a-zA-Z]', '', regex=True)\
                                         .str.replace(' ', '_')\
                                         .str.lower()

In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [135]:
df['adjusted_gre'] = df['grescore']

df.loc[df['universityrating'] >= 4, 'adjusted_gre'] += 10

df['adjusted_gre'] = pd.cut(df['adjusted_gre'], bins=4)